In [2]:
pip install xmltodict

  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install feedparser

  Using cached feedparser-6.0.10-py3-none-any.whl (81 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=ec72df405c928e7aab856b2fc225b6274b5f778edbb46a5edbcc8958b7e85c20
  Stored in directory: c:\users\lg\appdata\local\pip\cache\wheels\65\7a\a7\78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=87ce42bd602757eb3feca27707941e10fde141fdeb44ed3ff698be07742bf098
  Stored in directory: c:\users\lg\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [152]:
pip install readability-lxml

Note: you may need to restart the kernel to use updated packages.


In [157]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import urllib.robotparser
import xmltodict
import requests
import feedparser
import os
import requests
import re
from bs4 import BeautifulSoup
from dateutil import parser
import os.path
import glob
import tensorflow as tf
import scrapy
import logging

### GPU 설정

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' : 
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

In [39]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1992569682849195549
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2258003559
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18228283311010859170
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


### 로이터 뉴스 

### URL 생성하기

In [2]:
# robots.txt해석
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.reuters.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.reuters.com/sitemap.xml')

True

In [3]:
rp.can_fetch('*', 'https:/www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml')

True

In [4]:
rp.can_fetch("*", "https://www.reuters.com/finance/stocks/option")

False

#### sitemap.xml 파일로 URL 획득

In [5]:
sitemap = xmltodict.parse(requests.get(
    'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml').text
)

In [6]:
# 파일을 다운로드 하기 전에 dict에 무엇이 있는지 확인해보기
urls = [url['loc'] for url in sitemap['urlset']['url']]
# 공간을 차지하지 않도록 맨 앞 URL 몇 개만 출력한다.
print("\n".join(urls[0:3]))

https://www.reuters.com/sports/baseball/dodgers-launch-5-homers-blasting-angels-2023-07-09/
https://www.reuters.com/sports/baseball/phils-likely-without-bryce-harper-close-out-first-half-vs-marlins-2023-07-09/
https://www.reuters.com/sports/baseball/rangers-turn-dane-dunning-search-series-win-vs-nationals-2023-07-09/


#### RSS에서 URL 획득

In [7]:
import feedparser
feed = feedparser.parse('http://web.archive.org/web/20200613003232if_/http://feeds.reuters.com/Reuters/worldNews')

In [8]:
[(e.title, e.link) for e in feed.entries]

[('Mexico City to begin gradual exit from lockdown on Monday',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/OQtkVdAqHos/mexico-city-to-begin-gradual-exit-from-lockdown-on-monday-idUSKBN23K00R'),
 ('Mexico reports record tally of 5,222 new coronavirus cases',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/Rkz9j2G7lJU/mexico-reports-record-tally-of-5222-new-coronavirus-cases-idUSKBN23K00B'),
 ('Venezuela supreme court to swear in new electoral council leaders, government says',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/cc3R5aq4Ksk/venezuela-supreme-court-to-swear-in-new-electoral-council-leaders-government-says-idUSKBN23J39T'),
 ("One-fifth of Britain's coronavirus patients were infected in hospitals: Telegraph",
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/1_7Wb0S_6-8/one-fifth-of-britains-coronavirus-patients-were-infected-in-hospitals-telegraph-idUSKBN23J382'),
 ('France to lift border controls for EU travellers on June 15',
  'http://feeds.reuters.com/~r/

In [9]:
# 실제 URL 뽑아보기
[e.id for e in feed.entries]

['https://www.reuters.com/article/us-health-coronavirus-mexico-city/mexico-city-to-begin-gradual-exit-from-lockdown-on-monday-idUSKBN23K00R?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-mexico/mexico-reports-record-tally-of-5222-new-coronavirus-cases-idUSKBN23K00B?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-venezuela-politics/venezuela-supreme-court-to-swear-in-new-electoral-council-leaders-government-says-idUSKBN23J39T?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-britain-hospitals/one-fifth-of-britains-coronavirus-patients-were-infected-in-hospitals-telegraph-idUSKBN23J382?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-france-borders/france-to-lift-border-controls-for-eu-travellers-on-june-15-idUSKBN23J385?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-brazil/brazils-covid-19-deaths-sur

### 데이터 다운로드

### 파이썬을 사용한 HTML 페이지 다운로드

In [10]:
%%time
output_directory = './data/'
s = requests.Session()
for url in urls[0:10] : 
    # URL의 마지막/뒤 부분을 가져와서 파일 이름으로 사용한다.
    file = url.split("/")[-1]
    # file_path = os.path.join(output_directory, file)
    
    r = s.get(url)
    if r.ok : 
        with open('./file', 'w+b') as f : 
            f.write(r.text.encode('utf-8'))
    else : 
        print('error with URL %s' % url)

CPU times: total: 0 ns
Wall time: 5.8 s


#### wget를 사용한 HTML 페이지 다운로드

In [11]:
with open('urls.txt', 'w+b') as f : 
    f.write('\n'.join(urls).encode('utf-8'))

### 반정형 데이터 추출

#### 정규 표현식을 사용한 추출

In [12]:
url = 'https://www.reuters.com/article/us-health-vaping-marijuana-idUSKBN1WG4KT'

# 마지막 / 뒤의 부분을 파일 이름으로 사용하기
file = url.split('/')[-1] + '.html'
r = requests.get(url)
with open(file, 'w+b') as f : 
    f.write(r.text.encode('utf-8'))

In [13]:
with open(file, "r", encoding='utf-8') as f:
    html = f.read()
    g = re.search(r'<title>(.*)</title>', html, re.MULTILINE|re.DOTALL)
    if g:
        print(g.groups()[0])

Banned in Boston: Without vaping, medical marijuana patients must adapt | Reuters


#### HTML 파서를 사용한 데이터 추출

In [14]:
WA_PREFIX = 'http://web.archive.org/web/20200118131624/'
html = s.get(WA_PREFIX + url).text

In [15]:
soup = BeautifulSoup(html, 'html.parser')
soup.select('h1.ArticleHeader_headline')

[<h1 class="ArticleHeader_headline">Banned in Boston: Without vaping, medical marijuana patients must adapt</h1>]

In [16]:
soup.h1

<h1 class="ArticleHeader_headline">Banned in Boston: Without vaping, medical marijuana patients must adapt</h1>

In [17]:
soup.h1.text

'Banned in Boston: Without vaping, medical marijuana patients must adapt'

In [18]:
soup.title.text

'\n                Banned in Boston: Without vaping, medical marijuana patients must adapt - Reuters'

##### 기사 텍스트 추출

In [19]:
soup.select_one('div.StandardArticleBody_body').text

'BOSTON (Reuters) - In the first few days of the four-month ban on all vaping products in Massachusetts, Laura Lee Medeiros, a medical marijuana patient, began to worry.\xa0 FILE PHOTO: An employee puts down an eighth of an ounce marijuana after letting a customer smell it outside the Magnolia cannabis lounge in Oakland, California, U.S. April 20, 2018. REUTERS/Elijah NouvelageThe 32-year-old massage therapist has a diagnosis of post-traumatic stress disorder (PTSD) from childhood trauma. To temper her unpredictable panic attacks, she relied on a vape pen and cartridges filled with the marijuana derivatives THC and CBD from state dispensaries. There are other ways to get the desired effect from  marijuana, and patients have filled dispensaries across the state in recent days to ask about edible or smokeable forms. But Medeiros has come to depend on her battery-powered pen, and wondered how she would cope without her usual supply of cartridges.  “In the midst of something where I’m on t

##### 이미지 캡션 추출

In [20]:
soup.select('div.StandardArticleBody_body figure img')

[<img aria-label="FILE PHOTO: An employee puts down an eighth of an ounce marijuana after letting a customer smell it outside the Magnolia cannabis lounge in Oakland, California, U.S. April 20, 2018. REUTERS/Elijah Nouvelage" src="//web.archive.org/web/20200118131624im_/https://s3.reutersmedia.net/resources/r/?m=02&amp;d=20191001&amp;t=2&amp;i=1435991144&amp;r=LYNXMPEF9039L&amp;w=20"/>,
 <img src="//web.archive.org/web/20200118131624im_/https://s3.reutersmedia.net/resources/r/?m=02&amp;d=20191001&amp;t=2&amp;i=1435991145&amp;r=LYNXMPEF9039M"/>]

In [21]:
# 이미지의 캡션에 관심이 있는 경우
soup.select('div.StandardArticleBody_body figcaption')

[<figcaption><div class="Image_caption"><span>FILE PHOTO: An employee puts down an eighth of an ounce marijuana after letting a customer smell it outside the Magnolia cannabis lounge in Oakland, California, U.S. April 20, 2018. REUTERS/Elijah Nouvelage</span></div></figcaption>,
 <figcaption class="Slideshow_caption">Slideshow<span class="Slideshow_count"> (2 Images)</span></figcaption>]

##### URL 추출

In [22]:
# 표준 URL : canonical
soup.find('link', {'rel':'canonical'})['href']

'http://web.archive.org/web/20200118131624/https://www.reuters.com/article/us-health-vaping-marijuana-idUSKBN1WG4KT'

##### 목록 정보(작성자) 추출

In [23]:
soup.find('meta', {'name':'Author'})['content']

'Jacqueline Tempera'

In [24]:
# 모든 작성자 추출하기
sel = "div.BylineBar_first-container.ArticleHeader_byline-bar div.BylineBar_byline span"
soup.select(sel)

[<span><a href="/web/20200118131624/https://www.reuters.com/journalists/jacqueline-tempera" target="_blank">Jacqueline Tempera</a>, </span>,
 <span><a href="/web/20200118131624/https://www.reuters.com/journalists/jonathan-allen" target="_blank">Jonathan Allen</a></span>]

In [25]:
# 작성자 이름만 추출
[a.text for a in soup.select(sel)]

['Jacqueline Tempera, ', 'Jonathan Allen']

##### 링크 텍스트 추출(섹션)

In [26]:
soup.select_one('div.ArticleHeader_channel a').text

'Health News'

##### 읽기 시간 추출

In [27]:
soup.select_one('p.BylineBar_reading-time').text

'6 Min Read'

##### ID속성 추출

In [28]:
soup.select_one('div.StandardArticle_inner-container')['id']

'USKBN1WG4KT'

##### 속성 추출
- 속성 정보 : 텍스트 끝에서 찾을 수 있음.

In [29]:
soup.select_one('p.Attribution_content').text

'Reporting Jacqueline Tempera in Brookline and Boston, Massachusetts, and Jonathan Allen in New York; Editing by Frank McGurty and Bill Berkrot'

##### 타임스탬프 추출

In [30]:
ptime = soup.find('meta', {'property' : 'og:article:published_time'})['content']
print(ptime)

2019-10-01T19:23:16+0000


In [31]:
# datetime객체로 변환하기
parser.parse(ptime)

datetime.datetime(2019, 10, 1, 19, 23, 16, tzinfo=tzutc())

#### 스파이더링(웹 크롤링)

In [141]:
# 'w+' : 열기 모드(읽기 쓰기 허용)
def download_archive_page(page) : 
    filename = 'page-%06d.html' % page

    if not os.path.isfile(filename) : 
        url = 'https://www.reuters.com/news/archive' + \
        '?view=page&page=%d&pageSize=10' % page
        r = requests.get(url)
        with open(filename, 'w+') as f : 
            f.write(r.text)

def parse_archive_page(page_file) : 
    with open(page_file, 'r') as f : 
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')
    hrefs = ['https://www.reuters.com' + a['href']
             for a in soup.select('article.story div.story-content a')]
    return hrefs

def download_article(url) : 
    # 기사 파일이 지정된 폴더에 이미 저장되었는지 확인
    filename = url.split('/')[-1] + '.html'
    if not os.path.isfile(filename) : 
        r = requests.get(url)
        with open(filename, 'w+') as f : 
            f.write(r.text)

def parse_article(article_file) : 
    with open(article_file, 'r') as f : 
        html = f.read()
    r = {}
    soup = BeautifulSoup(html, 'html.parser')
    try : 
        r['url'] = soup.find('link', {'rel':'canonical'})['href']
        r['id'] = r['url'].split('-')[-1]
        r['headline'] = soup.h1.text
        r['text'] = soup.select_one('div.StandardArticleBody_body').text
        r['section'] = soup.select_one("div.ArticleHeader_channel a").text
        r['authors'] = [a.text for i in soup.select(sel)]
        r['time'] = soup.find("meta", { 'property': "og:article:published_time"})['content']
        return r
    except AttributeError as e : 
        return {'url':'','id':'','headline':'','text':'','section':'','authors':'','time':''}


In [61]:
# download 10 pages of archive
for p in range(1, 10):
    download_archive_page(p)

In [62]:
# 아카이브를 분석하고 article_urls에 추가하기
article_urls = []
for page_file in glob.glob("page-*.html"):
    article_urls += parse_archive_page(page_file)

In [63]:
# 기사 다운도르 하기
for url in article_urls:
    download_article(url)

In [147]:
# 판다스의 데이터프레임에서 배치함
df = pd.DataFrame()
for article_file in glob.glob("*-id???????????.html"):
    # df = df.append(parse_article(article_file), ignore_index=True)
    # df_new_row = pd.DataFrame(parse_article(article_file))
    # df = pd.concat(df, df_new_row, ignore_index=True)
    df = pd.concat([df, parse_article(article_file)], ignore_index=True)
df['time'] = pd.to_datetime(df.time)

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

### 밀도 기반 텍스트 추출

#### 리더빌리티로 로이터 콘텐츠 추출

In [153]:
from readability import Document

doc = Document(html)
doc.title()

'Banned in Boston: Without vaping, medical marijuana patients must adapt - Reuters'

In [154]:
doc.short_title()

'Banned in Boston: Without vaping, medical marijuana patients must adapt'

In [155]:
# 실제 콘텐츠에서 확인
doc.summary()

'<html><body><div><div class="StandardArticleBody_body"><p>BOSTON (Reuters) - In the first few days of the four-month ban on all vaping products in Massachusetts, Laura Lee Medeiros, a medical marijuana patient, began to worry.\xa0 </p><div class="PrimaryAsset_container"><div class="Image_container" tabindex="-1"><figure class="Image_zoom"></figure><figcaption><p class="Image_caption"><span>FILE PHOTO: An employee puts down an eighth of an ounce marijuana after letting a customer smell it outside the Magnolia cannabis lounge in Oakland, California, U.S. April 20, 2018. REUTERS/Elijah Nouvelage</span></p></figcaption></div></div><p>The 32-year-old massage therapist has a diagnosis of post-traumatic stress disorder (PTSD) from childhood trauma. To temper her unpredictable panic attacks, she relied on a vape pen and cartridges filled with the marijuana derivatives THC and CBD from state dispensaries. </p><p>There are other ways to get the desired effect from  marijuana, and patients have 

In [156]:
density_soup = BeautifulSoup(doc.summary(), 'html.parser')
density_soup.body.text

'BOSTON (Reuters) - In the first few days of the four-month ban on all vaping products in Massachusetts, Laura Lee Medeiros, a medical marijuana patient, began to worry.\xa0 FILE PHOTO: An employee puts down an eighth of an ounce marijuana after letting a customer smell it outside the Magnolia cannabis lounge in Oakland, California, U.S. April 20, 2018. REUTERS/Elijah NouvelageThe 32-year-old massage therapist has a diagnosis of post-traumatic stress disorder (PTSD) from childhood trauma. To temper her unpredictable panic attacks, she relied on a vape pen and cartridges filled with the marijuana derivatives THC and CBD from state dispensaries. There are other ways to get the desired effect from  marijuana, and patients have filled dispensaries across the state in recent days to ask about edible or smokeable forms. But Medeiros has come to depend on her battery-powered pen, and wondered how she would cope without her usual supply of cartridges.  “In the midst of something where I’m on t

### 스크래피 활용하기